In [ ]:
# NEXT:
# analysis
# run similarity check

In [2]:
import pandas as pd
import numpy as np
import csv

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 15000)

In [3]:
tdb = pd.read_pickle('../workproduct-files/cleaned-dataframes/tdb.pkl')
opentdb = pd.read_pickle('../workproduct-files/cleaned-dataframes/opentdb.pkl')
tfyi = pd.read_pickle('../workproduct-files/cleaned-dataframes/tfyi.pkl')
rtg = pd.read_pickle('../workproduct-files/cleaned-dataframes/rtg.pkl')

In [4]:
# Define columns to be used in master data
cols_orig = ['ORIG_id', 'ORIG_question', 'ORIG_answer', 'ORIG_alt answers', 'ORIG_category', 'ORIG_alt categories', 'ORIG_difficulty', 'ORIG_type']
cols_cons = ['CONS_id', 'CONS_question', 'CONS_answer', 'CONS_alt answers', 'CONS_category', 'CONS_alt categories - NOT USED', 'CONS_type-formulation', 'CONS_type-multipleChoice']
cols_cons.extend(cols_orig)
cols = cols_cons


In [5]:
# Help info only
print(f'tdb: {tdb.columns.values}\n')
print(f'opentdb: {opentdb.columns.values}\n')
print(f'tfyi: {tfyi.columns.values}\n')
print(f'rtg: {rtg.columns.values}\n')

tdb: ['category_id' 'lang' 'tags' 'question' 'answer' 'answers' 'source']

opentdb: ['category' 'type' 'difficulty' 'question' 'correct_answer'
 'incorrect_answers']

tfyi: ['question' 'answer' 'categories' 'primary_category']

rtg: ['_id' 'question_no_a' 'question' 'answer' 'user' 'categories' 'tags'
 'created' 'question_no_h' 'question_no_s' 'question_no_m' 'question_no_g'
 'question_no_e']



### Create master table for each data set

In [6]:
# tdb

cons_tdb = pd.DataFrame(columns = cols)

# Format original id to hexadecimal with padding 8
cons_tdb['ORIG_id'] = tdb.index.values
cons_tdb['ORIG_id'] = cons_tdb['ORIG_id'].apply(lambda x: f"tdb_{x:#0{8}x}")

cons_tdb['ORIG_category'] = tdb['category_id']
cons_tdb['ORIG_question'] = tdb['question']
cons_tdb['ORIG_answer'] = tdb['answer']
cons_tdb['ORIG_alt answers'] = tdb['answers']

cons_tdb['CONS_question'] = cons_tdb['ORIG_question']
cons_tdb['CONS_answer'] = cons_tdb['ORIG_alt answers'].apply(lambda x: x[0])

# NOTE: 'CONS_alt answers' should contain only incorrect answers
cons_tdb['CONS_alt answers'] = cons_tdb['ORIG_alt answers'].apply(lambda x: x[1:]) # index 0 is removed (correct answer)
cons_tdb.loc[cons_tdb['CONS_alt answers'].map(len) == 0, ['CONS_alt answers']] = np.nan # empty lists are changed to NaN

In [7]:
# opentdb

cons_opentdb = pd.DataFrame(columns = cols)

# Format original id to hexadecimal with padding 8
cons_opentdb['ORIG_id'] = opentdb.index.values
cons_opentdb['ORIG_id'] = cons_opentdb['ORIG_id'].apply(lambda x: f"opentdb_{x:#0{8}x}")

cons_opentdb['ORIG_category'] = opentdb['category']
cons_opentdb['ORIG_question'] = opentdb['question']
cons_opentdb['ORIG_answer'] = opentdb['correct_answer']
cons_opentdb['ORIG_alt answers'] = opentdb['incorrect_answers']
cons_opentdb['ORIG_difficulty'] = opentdb['difficulty']
cons_opentdb['ORIG_type'] = opentdb['type']

cons_opentdb['CONS_question'] = cons_opentdb['ORIG_question']
cons_opentdb['CONS_answer'] = cons_opentdb['ORIG_answer']

# NOTE: 'CONS_alt answers' should contain only incorrect answers
cons_opentdb['CONS_alt answers'] = cons_opentdb['ORIG_alt answers']

In [8]:
# tfyi

def shortenText(x):
    y = []
    for item in x:
        y.append(item[:-7])
    x = y
    return x


cons_tfyi = pd.DataFrame(columns = cols)

cons_tfyi['ORIG_id'] = tfyi.index.values
cons_tfyi['ORIG_id'] = cons_tfyi['ORIG_id'].apply(lambda x: f"tfyi_{x:#0{8}x}")

cons_tfyi['ORIG_category'] = tfyi['primary_category'].apply(lambda x: x[:-7]) # string ' Trivia' is removed from end of category name
cons_tfyi['ORIG_question'] = tfyi['question']
cons_tfyi['ORIG_answer'] = tfyi['answer']
cons_tfyi['ORIG_alt categories'] = tfyi['categories']
cons_tfyi['ORIG_alt categories'] = cons_tfyi['ORIG_alt categories'].apply(lambda x: x[1:]) # index 0 is removed (primary category)
cons_tfyi['ORIG_alt categories'] = cons_tfyi['ORIG_alt categories'].apply(lambda x: shortenText(x)) # string ' Trivia' is removed from end of category name
cons_tfyi.loc[cons_tfyi['ORIG_alt categories'].map(len) == 0, ['ORIG_alt categories']] = np.nan # empty lists are changed to NaN

cons_tfyi['CONS_question'] = cons_tfyi['ORIG_question']
cons_tfyi['CONS_answer'] = cons_tfyi['ORIG_answer']

In [9]:
# rtg

cons_rtg = pd.DataFrame(columns = cols)

cons_rtg['ORIG_id'] = rtg.index.values
cons_rtg['ORIG_id'] = cons_rtg['ORIG_id'].apply(lambda x: f"rtg_{x:#0{8}x}")

cons_rtg['ORIG_category'] = rtg['categories']
cons_rtg['ORIG_question'] = rtg['question']
cons_rtg['ORIG_answer'] = rtg['answer']
cons_rtg['ORIG_alt categories'] = rtg['tags']
cons_rtg.loc[cons_rtg['ORIG_alt categories'].map(len) == 0, ['ORIG_alt categories']] = np.nan # empty lists are changed to NaN

cons_rtg['CONS_question'] = cons_rtg['ORIG_question']
cons_rtg['CONS_answer'] = cons_rtg['ORIG_answer']

### Creating consolidated master table

In [10]:
# Combining to master

t_data = cons_tdb.append([cons_opentdb, cons_tfyi, cons_rtg])
t_data.reset_index(drop = True, inplace = True)

In [11]:
# Strip leading and trailing whitespaces in 'CONS_question' and 'CONS_answer'

t_data['CONS_question'] = t_data['CONS_question'].str.strip()
t_data['CONS_answer'] = t_data['CONS_answer'].str.strip()

In [12]:
# Import category mapping file

with open('../edited-data/Category mapping.csv', mode='r') as infile:
    reader = csv.reader(infile)
    catMapping = {rows[0]:rows[1] for rows in reader}

In [13]:
# Map values in 'ORIG_category' to 'CONS_category' using category mapping table
# NOTE: 'CONS_alt categories' not mapped and filled - column not used

t_data['CONS_category'] = t_data['ORIG_category']
t_data['CONS_category'].replace(catMapping, inplace = True)

In [14]:
# If alternative answers exist, 'CONS_type-multipleChoice' is set to True, otherwise False

t_data['CONS_type-multipleChoice'] = t_data['CONS_alt answers'].notnull()

In [15]:
# 'CONS_type-formulation'
# TODO: If answer is 'True' or 'False' -> set 'CONS_type-formulation' to 'Statement - boolean'

In [16]:
#Correcting 'CONS_question' formatting (removing trailing " and adding ? where necessary)

# Find questions that end in " and have an odd number of " (i.e. quotes are not part of the question)
qEndinQuote = t_data.loc[:,'CONS_question'].apply(lambda x: x.count('"') % 2 != 0 and x[-1] in ['"'])

# From these, remove the trailing "
t_data.loc[qEndinQuote, 'CONS_question'] = t_data.loc[qEndinQuote, 'CONS_question'].apply(lambda x: x[:-1])

# Find questions containing question word and not ending in ? or . or ?"
questionWords = ('who', 'what', 'when', 'where', 'why', 'how', 'which')
qNeedsQmark = t_data.loc[:,'CONS_question'].apply(lambda x: any(y in x.lower() for y in questionWords) and x[-1] not in ['?', '.'] and x[-2:] not in ['?"'])

# Add ? to end if necessary
t_data.loc[qNeedsQmark, 'CONS_question'] = t_data.loc[qNeedsQmark, 'CONS_question'].apply(lambda x: x + '?')

In [17]:
# Add 'CONS_type-formulation' value: 'Question', 'Statement - open', 'Statement - boolean'

endInQmark = t_data['CONS_question'].apply(lambda x: x[-1] == '?' or x[-2:] == '?"')
answerTrueOrFalse = t_data['CONS_answer'].apply(lambda x: x.lower() == 'true' or x.lower() == 'false')

# Set all 'CONS_type-formulation' to 'Statement - open'
t_data['CONS_type-formulation'] = 'Statement - open'

# Set rows where question ends in ? to 'Question' (overrides previous)
t_data.loc[endInQmark, 'CONS_type-formulation'] = 'Question'

# Set rows where answer is true or false to 'Statement - boolean' (overrides previous)
t_data.loc[answerTrueOrFalse, 'CONS_type-formulation'] = 'Statement - boolean'

In [18]:
# Set 'CONS_id' to 'ORIG_id'

t_data['CONS_id'] = t_data['ORIG_id']

In [19]:
# Add 'Source' column to t_data and set value from 'ORIG_id'

t_data['Source'] = t_data['ORIG_id'].apply(lambda x: x.split('_')[0])

In [20]:
t_data.to_pickle('../workproduct-files/cleaned-dataframes/t_dataMaster.pkl')
# print(t_data)